In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!
import ollama

In [ ]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:

system_message = "You are a assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1/', api_key='OLLAMA')

gemini_via_openai = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
                           api_key = google_api_key)

In [ ]:
def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama_via_openai.chat.completions.create(
        model='llama3.2',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = gemini_via_openai.chat.completions.create(
        model='gemini-1.5-flash',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_brochure(company_name, url, model, response_tone):
    prompt = f"You are a professional marketing copywriter. Please generate a {response_tone} company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Ollama":
        result = stream_ollama(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

### Option 2: using prompt templating

In [ ]:
PROMPT_TEMPLATES = {
    "Informational": lambda company, content: (
        f"Write an informational brochure for {company}. Include:\n"
        f"1. Company Overview\n2. Products/Services\n3. Industry Expertise\n4. Contact Info\n\n"
        f"Use this content:\n{content}"
    ),
    "Promotional": lambda company, content: (
        f"You're a creative marketing expert. Write a persuasive, promotional brochure for {company} "
        f"emphasizing unique selling points, customer benefits, and a strong call to action.\n\n"
        f"Use this landing page content:\n{content}"
    ),
    "Humorous": lambda company, content: (
        f"Create a witty and humorous brochure for {company}. Keep it fun and light while still highlighting "
        f"what the company does. Be quirky but clear.\n\nContent:\n{content}"
    )
}


In [ ]:
def stream_brochure_template(company_name, url, model, response_tone):
    content = Website(url).get_contents()
    if not content.strip():
        raise ValueError("Website content is empty or could not be fetched.")

    try:
        prompt = PROMPT_TEMPLATES[response_tone](company_name, content)
    except KeyError:
        raise ValueError("Invalid tone selected.")

    model_map = {
        "GPT": stream_gpt,
        "Ollama": stream_ollama,
        "Gemini": stream_gemini
    }
    try:
        yield from model_map[model](prompt)
    except KeyError:
        raise ValueError("Unknown model selected.")


In [ ]:
view = gr.Interface(
    fn=stream_brochure_template,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Ollama", "Gemini"], label="Select model"),
        gr.Dropdown(["Informational", "Promotional", "Humorous"], label="Select tone")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()